In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def SE_block(input_tensor, ratio=16):
    # Squeeze operation
    squeezed = layers.GlobalAveragePooling2D()(input_tensor)
    
    # Excitation operation
    excitation = layers.Dense(units=input_tensor.shape[-1] // ratio, activation='relu')(squeezed)
    excitation = layers.Dense(units=input_tensor.shape[-1], activation='sigmoid')(excitation)
    excitation = layers.Reshape((1, 1, -1))(excitation)
    scaled_input = layers.Multiply()([input_tensor, excitation])
    
    return scaled_input

# Define the convolutional neural network architecture
def CNN_with_SE(input_shape, num_classes):
    model = models.Sequential()

    # Convolutional layers with SE block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(SE_block(model.output))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(SE_block(model.output))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(SE_block(model.output))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten layer
    model.add(layers.Flatten())
    
    # Fully connected layers
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Create an instance of the CNN with SE block
input_shape = (64, 64, 3)  # Example input shape
num_classes = 10  # Example number of classes
model = CNN_with_SE(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()
